In [1]:
import pandas as pd 
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.runnables import chain
from bs4 import BeautifulSoup
import requests
from pydantic import BaseModel, Field

In [2]:
url_dataset = '../../datasets/relevantes/dataset_links.csv'

dataset = pd.read_csv(url_dataset)

dataset.head()

,link
0,https://bit.ly/3lUcz1t
1,https://bit.ly/3g4FlsE
2,https://bit.ly/3g7mn4w
3,https://bit.ly/3sjQ0UY
4,https://bit.ly/3m6uVwh


In [3]:
dataset['link'][0]

'https://bit.ly/3lUcz1t'

# na mão

In [18]:
def fetch_page_content(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Checa se houve erro na resposta
        soup = BeautifulSoup(response.content, "html.parser")
        text = soup.get_text(separator="\n").strip()
        return text if text else "Conteúdo vazio ou indisponível."
    except Exception as e:
        return f"Erro ao acessar o link: {e}"

In [19]:
class Response(BaseModel):
    summary: str = Field(description="Resumo da notícia fornecida.", required=True)

In [20]:
summary_prompt = PromptTemplate(
    input_variables=["content"],
    template="""
Você é um assistente especializado em gerar resumos detalhados.

Seu objetivo é fornecer um resumo claro e completo, destacando os principais pontos, argumentos e informações relevantes do conteúdo. O resumo deve ser informativo e bem estruturado, com foco nas ideias principais. 

Lembre-se de que o resumo deve ser coeso e coerente, respeitando um limite máximo de 350 palavras, para permitir um pouco mais de profundidade e detalhes.

Texto:
{content}
"""
)

llm = ChatOllama(model="llama3.1", format="json", temperature=0.1)

structured_llm = llm.with_structured_output(Response)

summary_chain = summary_prompt | structured_llm


In [21]:
def summarize_web_content(url):
    page_content = fetch_page_content(url)
    
    if "Erro" in page_content or "Conteúdo vazio" in page_content:
        return page_content  
    
    summary = summary_chain.invoke(page_content)
    return summary


In [22]:
url = dataset['link'][0]
response = summarize_web_content(url)

In [23]:
response

'Queimada atinge casa de festas e empresário estima R$ 80 mil de prejuízo - Capital - Campo Grande News\n \n \n\n\n \n \n \n\n\n\n\n\n\nmenu\n\n\nMenu\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nACOMPANHE-NOS\n\n\xa0\xa0\xa0\n\n\n\n\n\n\n\n (67) 99669-9563\n\n\n \nDEZEMBRO, QUARTA\n \xa0\n18\n\n\xa0\xa0\n\xa0\nCAMPO GRANDE\n\xa0\n29º\n\n\n\n\nsearch\nBusca\n\n\n\n\n\n\n\n\nAs mais\n \nMais\n lidas\n\n\n\n\nMotociclista de aplicativo morre ao ser atingido por moto de alta cilindrada\nEmpresário "acorrenta" carro que parou em vaga exclusiva para clientes\nAcidente mata 2 adultos, 1 criança e deixa 3 feridos\nAcidente que matou motociclista de aplicativo foi gravado por câmeras \nMais uma gigante da laranja deve se instalar na costa leste do Estado\n\n\nÚltimas notícias\nCapital\nQueimada atinge casa de festas e empresário estima R$ 80 mil de prejuízo\nFogo teve início em terrenos próximos do local e se espalhou até espaço de eventos\n\nAletheya Alves e Cristiano Arruda | 10/08/2021 17:08\n\n\n\

# usando biblioteca  

In [13]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

page_url = "https://bit.ly/3lUcz1t"

loader = WebBaseLoader(web_paths=[page_url])
docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

assert len(docs) == 1
doc = docs[0]

In [23]:
print(f"{doc.metadata}\n")
print(doc.page_content[:500].strip())
# print(doc.page_content[:500].strip())

{'source': 'https://bit.ly/3lUcz1t', 'title': 'Queimada atinge casa de festas e empresário estima R$ 80 mil de prejuízo - Capital - Campo Grande News', 'description': 'Rotineiro durante época de seca, incêndio em vegetação se espalhou e destruiu uma casa de festas nas Moreninhas durante a tarde desta terça-feira (10). Ap...', 'language': 'pt-br'}

Queimada atinge casa de festas e empresário estima R$ 80 mil de prejuízo - Capital - Campo Grande News  
   


menu
Menu








ACOMPANHE-NOS
   



 (67) 99669-9563
 DEZEMBRO, QUARTA  11
   CAMPO GRANDE 33º

searchBusca



As mais Mais lidas

Espetaria usava corante em carne, armazenada entre moscas e baratasGrupo decide "fazer justiça" e espanca suspeito de furto no GuanandiPara surpresa de Ayache, filiados à Cassems votaram por manter eleição ilimitadaPM confunde celular com arma e atira e


In [9]:
loader = WebBaseLoader(
    web_paths=[page_url],
    bs_kwargs={
        "parse_only": bs4.SoupStrainer(class_="theme-doc-markdown markdown"),
    },
    bs_get_text_kwargs={"separator": " | ", "strip": True},
)

docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

assert len(docs) == 1
doc = docs[0]

In [10]:
print(f"{doc.metadata}\n")
print(doc.page_content[:500])

{'source': 'https://bit.ly/3lUcz1t'}




In [4]:
from langchain_unstructured import UnstructuredLoader

page_url = "https://siscad.ufms.br/"

loader = UnstructuredLoader(web_url=page_url)

docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

In [5]:
for doc in docs[:5]:
    print(doc.page_content)

Bem-vindo ao Portal de Graduação!
Para acessar o sistema acadêmico, clique no botão abaixo:
Acessar o Siscad
Para outros você pode conferir abaixo:
Bem-vindo ao Portal de Graduação!


In [6]:
for doc in docs[:5]:
    print(f'{doc.metadata["category"]}: {doc.page_content}')

Title: Bem-vindo ao Portal de Graduação!
Title: Para acessar o sistema acadêmico, clique no botão abaixo:
Title: Acessar o Siscad
NarrativeText: Para outros você pode conferir abaixo:
Title: Bem-vindo ao Portal de Graduação!


 # Refazendo scraping